In [1]:
!pip install folium
!pip install pandas


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [folium]


In [2]:

import folium
import pandas as pd
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

In [3]:
# Download and read the `spacex_launch_geo.csv`

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'

spacex_df=pd.read_csv(URL)

In [4]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


In [5]:
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)
site_map

In [6]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

In [7]:
folium.map.Marker(nasa_coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
site_map


In [8]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


In [10]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label
marker_cluster = MarkerCluster()
def get_marker_color(launch_class):
    if launch_class == 1:
        return 'green'
    else:
        return 'red'

# Apply the function to the 'class' column and create the new column
spacex_df['marker_color'] = spacex_df['class'].apply(get_marker_color)

# 2. Add the empty marker cluster layer to your map (you can do this first or last)
site_map.add_child(marker_cluster)

# 3. Iterate through the spacex_df DataFrame
for index, record in spacex_df.iterrows():
    # Get the coordinates and color from the current row
    current_coordinate = [record['Lat'], record['Long']]
    current_color = record['marker_color'] # 'green' or 'red'
    popup_text = f"{record['Launch Site']}"

    # 4. Create a Marker object with its coordinate and customized icon property
    marker = folium.Marker(
        location=current_coordinate,
        popup=popup_text,
        icon=folium.Icon(
            color='white',         # The background color of the icon base
            icon_color=current_color, # The color of the specific icon inside (like a star or info-sign)
            icon='info-sign'       # A standard icon name
        )
    )

    # 5. Add the marker to the marker cluster
    marker_cluster.add_child(marker)

# Display the map (assuming you are in a Jupyter environment)
site_map

In [11]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

In [18]:
lat = 28.56299
lon = -80.57736
coastline_lat = 28.56521
coastline_lon = -80.56772
CCAFS_LC_40= [lat,lon]


from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

distance_coastline = calculate_distance(lat,lon, coastline_lat, coastline_lon)
print(f'{distance_coastline:.2f} KM')

0.97 KM


In [19]:
lat = 28.56299
lon = -80.57736
coastline_lat = 28.56521
coastline_lon = -80.56772
coastline_coordinate = [coastline_lat,coastline_lon]
CCAFS_LC_40= [lat,lon]


# Create a marker for the coastline point with the distance label
distance_marker = folium.Marker(
    CCAFS_LC_40,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_coastline),
    )
)

# Add the distance marker to the map
site_map.add_child(distance_marker)

# Create a `folium.PolyLine` object using the two coordinates
coordinates = [CCAFS_LC_40,coastline_coordinate]
lines = folium.PolyLine(
    locations=coordinates, 
    weight=1, 
    color='blue'
)

# Add the PolyLine to the map
site_map.add_child(lines)

In [24]:
def calculate_distance(lat1, lon1, lat2, lon2):
    """Calculates distance in kilometers (Haversine formula)."""
    R = 6371.0
    lat1_rad, lon1_rad, lat2_rad, lon2_rad = map(radians, [lat1, lon1, lat2, lon2])
    dlon = lon2_rad - lon1_rad
    dlat = lat2_rad - lat1_rad
    a = sin(dlat / 2)**2 + cos(lat1_rad) * cos(lat2_rad) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = R * c
    return distance

# Define the single launch site coordinate
launch_site_name = 'CCAFS SLC-40'
launch_site_coords = [28.563197, -80.576820]

# Define proximity points with names and colors for visualization
proximities = {
    'Coastline': ([28.56367, -80.57163], 'blue'),
    
    'Railway (FEC)': ([28.5721, -80.8526], 'red'),
    'City (Titusville)': ([28.52361, -80.8062], 'purple')
}

# Initialize the map centered on the launch site
site_map = folium.Map(location=launch_site_coords, zoom_start=11)

# Add a marker for the launch site itself
folium.Marker(
    location=launch_site_coords,
    popup=f'<b>{launch_site_name}</b>',
    icon=folium.Icon(color='black', icon='rocket')
).add_to(site_map)


# Iterate through each proximity point to calculate distance and plot
for prox_name, (prox_coords, color) in proximities.items():
    prox_lat, prox_lon = prox_coords
    site_lat, site_lon = launch_site_coords
    
    # Calculate the distance
    distance = calculate_distance(site_lat, site_lon, prox_lat, prox_lon)
    
    # Create the distance marker at the proximity point
    distance_marker = folium.Marker(
        prox_coords,
        icon=DivIcon(
            icon_size=(250, 20),
            icon_anchor=(0, 0),
            html=f'<div style="font-size: 12px; color:{color};"><b>{prox_name}: {distance:.2f} KM</b></div>'
        )
    )
    site_map.add_child(distance_marker)
    
    # Create the PolyLine object
    lines = folium.PolyLine(
        locations=[launch_site_coords, prox_coords], 
        weight=2,
        color=color
    )
    site_map.add_child(lines)

# Display the map in the output/notebook
site_map